# R-Studio Data Wrangling and Cleaning:

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)


# Purpose & Outcome:

+ Learn Different Techniques to manipulate your data

+ Extend your skills and learn something useful, and remember: `always, be leery of steely-eyed orange guys with weird hair and fake news.`

**Disclaimer**: always, evaluate your data and look at examples as just that; and NOT a savior to all your problems. These are tools to aid you NOT answer every concernt that comes up, because they are all different. 


In [1]:
library(tidyverse)
library(knitr) # 


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Sales Data With Removed Canceled Orders:
 

In [ ]:
sales_orders<-read.csv('SalesDataNoCancels.csv')

# head(sales_orders)

# head(subset(sales_orders$Country!='United Kingdom'))
# sales_wo_uk<-subset(sales_orders, Country!='United Kingdom')

In [ ]:
# One Way To Change to Date Format:
date_t_01<-sales_orders%>% mutate_at(vars(InvoiceDate), as.Date, format="%m/%d/%Y %H:%M")
head(date_t_01)

In [ ]:
# the UK dwarfs the data, that is why I remove it here:
sales_wo_uk<-subset(sales_orders, Country!='United Kingdom')

# Parsing the dates from the data for our examples today.

In [ ]:
# Taking the Date column and converting from char -> date/time
time_<-strptime(sales_wo_uk$InvoiceDate, "%m/%d/%Y %H:%M")

time_conversion<-as.POSIXct(time_) # convert object to class: date/time

dates_fix<-data.frame(
   date=time_conversion,
   hour=format(time_conversion, "%H"),
   day=format(time_conversion,"%d"),
   month=format(time_conversion,"%m"),
   year=format(time_conversion,"%Y"),
month_year=format(time_conversion,"%m/%Y"))



head(dates_fix)

In [ ]:
# Month Name:
dates_fix$month_abbrv<-month.abb[as.numeric(dates_fix$month)]


# Day of Week Names
dates_fix$day_of_week<-strftime(dates_fix$date,'%A')
head(dates_fix)

# Create a new column of `Sales totals=Qty*Price`

+ This is just 1 way of doing this by the way

In [ ]:

sales_wo_uk$Sales_Tot<-sales_wo_uk$Quantity*sales_wo_uk$UnitPrice
head(sales_wo_uk)

# If you had to combine two data frames:

+ There is `cbind`: which works columnwise, meaning it forms an abuttment.
    + Otherwise, you have `rbind`: which would just append to end of rows

In [ ]:
sales_orders_expanded<-cbind(sales_wo_uk,dates_fix)

head(sales_orders_expanded)

# Change Layout of Data :

For the first few examples, lets evaluate these data as `CustomerID`, `SalesTot`, `Monthyear`
to create a new table

In [ ]:
sub<-c('CustomerID','Sales_Tot','day_of_week')
head(sales_orders_expanded[sub])

# *Reshape*: using `tidyr` which is found in `tidyverse`

`----------------------------------------------------`

# `Gather`: Wide to Long




In [ ]:
mean_<-c(60,50,40,30,10)
sd_<-c(2,1,3,2,4)
items<-c('a','a','b','c','d')
df_1<-data.frame(items,mean_,sd_)

df_1

gather(df_1,mean_,sd_,key='keys',value='vals')

# `separate`: Make Long Data Wide

+ spliting 1 column into multiple columns

In [ ]:
# `separate`: Make Long Data Wide, spliting 1 column into multiple columns


# sub_<-c('Country','CustomerID','Sales_Tot','day_of_week')
# head(sales_orders_expanded[sub_])
long_to_wide<-separate(sales_orders,'InvoiceDate',c('month','day','Year','hour','min'))

head(long_to_wide)

# `Unite`: make 2 variables into 1

+ creating a time of hour and minutes together

if you do not put a separator, by default you will have an underscore 

In [ ]:
long_to_wide %>% unite(Time,hour,min,sep=':')

# `Seperate_rows`: Convert 1 column into more rows from an invidual cell


In [ ]:
shoe_types<-c('trail, street,hybrid','track,sandals','walking','high heels',
              -'pumps','boots')
arch_type<-c('high','low','flat','medium','extra crazy','who knows')
size_s<-c(9,10,11,12,13,7)

shoes_stuff<-data.frame(shoe_types,arch_type,size_s)
shoes_stuff

In [ ]:
shoes_stuff %>% separate_rows(shoe_types,sep=',')

#  `Spread`:  separates each cell into a column

Think of using a two columns: one as a key, other as a values. then you take the column that was a key and expand that into more columns based on unique row values. 

In [ ]:
ww<-shoes_stuff %>% separate_rows(shoe_types,sep=',')
ww%>% spread(shoe_types,size_s) 

# `Complete`: takes a set of cloumns and finds all combinations that are unique. *It will fill in NA values  when neccessary*.

In [ ]:
sub_w<-c('CustomerID','Sales_Tot','month_year')
a<-as.factor(sales_orders_expanded[sub_w]$month_year)
cc<-sales_orders_expanded[sub_w]
cc$date_col_asfactor<-a

complete(data = head(cc,10 ),date_col_asfactor)

# Notice that the datatypes 'classes' are not the same

In [ ]:
# read.csv('online_sales.csv')

# What is `Dplyr`: it is a package inside `tidyverse` allowing us to manipulate and transform our data

+ As an artifact you are able to write code that is readable for humans and cogent. 
+ There are 5 very common functions ("verbs") that you will use when using `dplyr`
    + `filter`
    + `select`
    + `arrange`
    + `mutate`
    + `summarise`
    


https://programminghistorian.org/en/lessons/data_wrangling_and_management_in_R

# Let's find the shoes in stock in a certain size range:

+ Using: `filter and select`

`filter`: works by row

`select`: columnwise

In [ ]:
shoes_long_<-shoes_stuff %>% separate_rows(shoe_types,sep=',')

shoes_by_size_range <- shoes_long_ %>%
  filter(size_s >9 & size_s <12) %>%
  select(shoe_types,arch_type,size_s)

shoes_by_size_range




# `Arrange`: will arrange rows by a specific column value

In [ ]:
# Going from descending order of shoes by size
shoes_long_ %>% arrange(desc(size_s))

In [ ]:
# Alt. Example with 2 variables:
head(date_t_01 %>% arrange(desc(Country),desc(InvoiceDate)))

# `Mutate`: Add a new variable to DF, think column

In [ ]:
head(date_t_01 %>% mutate(SalesTotsYay=Quantity*UnitPrice))

# `Summarize`: 

Creates a new dataframe that will have one or more rows for grouping, you are using a function to count, calculate the mean etc as a new column.

https://dplyr.tidyverse.org/reference/summarise.html

In [ ]:


ss<-shoes_long_   %>% 
  tibble::as_tibble() 

# count number of shoes based on size
ss %>%
  group_by(size_s) %>%
  summarise( n = n())

ss # print just the original data

# Group by shoe size and arch_type then count numbers available
ss %>%
  group_by(size_s,arch_type) %>%
  summarise( n = n())

# `Nest & Chop`: 

+ `Nest`: creates a list of column dataframes
    + There is a reverse of this called `unnest`
    
+ `Chop`: preserve the width of a DF, it is converting row lists into columns
    
https://tidyr.tidyverse.org/reference/chop.html

In [ ]:
# Originsl data:
ss
# Nest:
ss %>% nest(size_arch = c(shoe_types, arch_type))

# Chop:
ss %>% chop( c(shoe_types, arch_type))


# Alt Example:

In [ ]:
df <- tibble(x = c(1, 1, 1, 2, 2, 3), y = 1:6, z = 6:1)
# Note that we get one row of output for each unique combination of
# non-nested variables
df %>% nest(data = c(y, z))
df %>% chop(c(y, z))
df

# https://tidyr.tidyverse.org/reference/nest.html (this example came from here)

# Pivot from Wide to Long: `pivot_longer`


In [ ]:
#starting data: wide format
wide_shoes<-ww%>% spread(shoe_types,size_s) 
wide_shoes

In [ ]:
#convert to long data with pivot_longer:
wide_shoes %>% pivot_longer(cols=2:10, names_to = "Quarter", values_to = "Delay")

# Going in reverse: Long to Wide using `pivot_wider`

In [ ]:
# shoes_stuff %>% separate_rows(shoe_types,sep=',')
long_shoes<-wide_shoes %>% pivot_longer(cols=2:10, names_to = "Footwear",
                                        values_to = "Sizes")

# pivot_wider: convert long->wide
long_shoes%>% pivot_wider( names_from = Footwear, values_from = Sizes) 


`----------------------------`

# <font color=red>LIKE</font>, Share &

# <font color=red>SUB</font>scribe

# Citations & Help:

# <font size=6>◔̯◔</font>

`Cheat Sheets`

https://rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf?utm_campaign=Data%2BElixir&utm_medium=web&utm_source=Data_Elixir_19#:~:text=Tidy%20Data%20%2D%20A%20foundation%20for,works%20as%20intuitively%20with%20R.&text=tidyr%3A%3Agather(cases%2C%20%22,4)%20Gather%20columns%20into%20rows

https://github.com/rstudio/cheatsheets/blob/master/data-import.pdf 

https://rstudio.com/resources/cheatsheets/

`Good Examples and documentation`

https://bookdown.org/mikemahoney218/IDEAR/data-wrangling.html

https://rstudio-pubs-static.s3.amazonaws.com/221386_a6b7054b6536462fb3ba49e0341142e5.html

https://uc-r.github.io/tidyr

http://biostat.mc.vanderbilt.edu/wiki/pub/Main/ColeBeck/datestimes.pdf

https://medium.com/analytics-vidhya/advanced-data-wrangling-in-r-4-f98693b92851